In [11]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [1]:
!pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 31.5 MB/s eta 0:00:00:00:0100:01


In [3]:
import os

path = os.listdir('/kaggle/input')[0]
print("Input path:", path)

Input path: llm-human-classification-data


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import pandas as pd

from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
from typing import List

In [5]:
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"GPU Name: {torch.cuda.get_device_name(0)}")
        print(f"Total memory: {torch.cuda.get_device_properties(0).total_memory / 1024 ** 2:.2f} MB")
        print(f"Allocated memory: {torch.cuda.memory_allocated(0) / 1024 ** 2:.2f} MB")
        print(f"Reserved memory: {torch.cuda.memory_reserved(0) / 1024 ** 2:.2f} MB")
        print(f"Free (unreserved) memory: {(torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0)) / 1024 ** 2:.2f} MB")
    else:
        print("CUDA not available.")

print_gpu_memory()

GPU Name: Tesla P100-PCIE-16GB
Total memory: 16269.25 MB
Allocated memory: 0.00 MB
Reserved memory: 0.00 MB
Free (unreserved) memory: 0.00 MB


In [6]:
dataset = pd.read_csv('/kaggle/input/llm-human-classification-data/LLM-Human Classification Data.csv')
dataset.shape

(29145, 2)

In [7]:
llm_dataset = dataset[dataset['generated'] == 1]
human_dataset = dataset[dataset['generated'] == 0]
print(llm_dataset.shape, human_dataset.shape)

llm_dataset = llm_dataset[:int((llm_dataset.shape[0])*0.05)]
human_dataset = human_dataset[:int((human_dataset.shape[0])*0.03)]
print(llm_dataset.shape, human_dataset.shape)

smaller_dataset = pd.concat([llm_dataset, human_dataset])
smaller_dataset = smaller_dataset.sample(frac=1)
smaller_dataset.shape

(11637, 2) (17508, 2)
(581, 2) (525, 2)


(1106, 2)

In [8]:
print(smaller_dataset['generated'].value_counts())

generated
1    581
0    525
Name: count, dtype: int64


In [9]:
smaller_dataset.head()

,text,generated
182,Limiting car usage can have several advantages...,1
751,"""America's love affair with it's vehicles seem...",0
801,Reducing the usage of cars in today's world co...,0
385,"[Your Name]\n[Your Address]\n[City, State, ZIP...",1
495,"[Your Name]\n[Your Address]\n[City, State, ZIP...",1


In [10]:
class PerturbationGenerator:
    def __init__(self, model_name: str = "t5-small"):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.tokenizer.model_max_length = 512
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def perturb(self, text: str, num_samples: int = 9) -> List[str]:
        inputs = self.tokenizer(
            f"paraphrase: {text}", return_tensors="pt", max_length=512, truncation=True
        ).to(self.device)

        outputs = self.model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            num_return_sequences=num_samples,
            do_sample=True,
            max_length=512,
        )
        
        return [self.tokenizer.decode(out, skip_special_tokens=True) for out in outputs]

In [11]:
class SourceModel:
    def __init__(self, model_name="gpt2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.model.eval()

        
    def get_log_prob(self, texts: List[str]) -> torch.Tensor:
        log_probs = []
        with torch.no_grad():
            for text in texts:
                inputs = self.tokenizer(text, return_tensors="pt", truncation=True).to(self.device)
                
                # Handle empty input
                if inputs["input_ids"].shape[1] == 0:
                    log_probs.append(0.0)
                    continue
                
                # Handle single-token edge case
                num_tokens = inputs["input_ids"].shape[1]
                if num_tokens == 1:
                    log_probs.append(0.0)
                    continue
                
                outputs = self.model(**inputs, labels=inputs["input_ids"])
                logits = outputs.logits
                labels = inputs["input_ids"]
                
                log_probs_seq = torch.log_softmax(logits, dim=-1)[:, :-1, :]
                log_probs_text = torch.gather(
                    log_probs_seq, 
                    dim=2, 
                    index=labels[:, 1:, None]
                ).squeeze(-1).sum(dim=1)

                normalized_log_prob = log_probs_text.item() / (num_tokens - 1)  
                log_probs.append(normalized_log_prob)
            
        # print(log_probs)    
        return torch.tensor(log_probs)

In [12]:
class ScoreCalculator:
    def __init__(self):
        self.perturb_model = PerturbationGenerator(model_name='t5-small')
        self.source_model = SourceModel(model_name='gpt2')

    def score(self, candidate_text):
        perturbations = self.perturb_model.perturb(text=candidate_text, num_samples=200)     #   Increase for better accuracy
        candidate_text_log_prob = self.source_model.get_log_prob([candidate_text])
        perturbations_log_prob = self.source_model.get_log_prob(perturbations)

        detection_score = candidate_text_log_prob - perturbations_log_prob.mean()
        return detection_score


In [13]:
sc = ScoreCalculator()
sentences = list(smaller_dataset['text'])
pred_scores = []
for num, sentence in enumerate(sentences, 1):
    pred_scores.append(sc.score(candidate_text=sentence).item())
    print(f"Detection Score for sentence {num} is: {pred_scores[num-1]:0.4f}")
    torch.cuda.empty_cache()

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Detection Score for sentence 1 is: 1.4925
Detection Score for sentence 2 is: 0.9193
Detection Score for sentence 3 is: 0.9531
Detection Score for sentence 4 is: 0.7727
Detection Score for sentence 5 is: 1.1324
Detection Score for sentence 6 is: 1.1604
Detection Score for sentence 7 is: 0.4272
Detection Score for sentence 8 is: 1.1788
Detection Score for sentence 9 is: 0.9612
Detection Score for sentence 10 is: 0.6684
Detection Score for sentence 11 is: 1.6174
Detection Score for sentence 12 is: 0.9444
Detection Score for sentence 13 is: 1.5346
Detection Score for sentence 14 is: 1.3359
Detection Score for sentence 15 is: 0.4728
Detection Score for sentence 16 is: 0.7908
Detection Score for sentence 17 is: 0.2086
Detection Score for sentence 18 is: 0.6544
Detection Score for sentence 19 is: 0.5293
Detection Score for sentence 20 is: 0.7288
Detection Score for sentence 21 is: 0.7866
Detection Score for sentence 22 is: 0.0996
Detection Score for sentence 23 is: 1.1404
Detection Score for 

In [15]:
print_gpu_memory()

GPU Name: Tesla P100-PCIE-16GB
Total memory: 16269.25 MB
Allocated memory: 495.18 MB
Reserved memory: 504.00 MB
Free (unreserved) memory: 8.82 MB


In [24]:
score_dataset = smaller_dataset.copy()
score_dataset["pred score"] = pred_scores
score_dataset.head()

,text,generated,pred score
182,Limiting car usage can have several advantages...,1,1.492491
751,"""America's love affair with it's vehicles seem...",0,0.919343
801,Reducing the usage of cars in today's world co...,0,0.953056
385,"[Your Name]\n[Your Address]\n[City, State, ZIP...",1,0.772685
495,"[Your Name]\n[Your Address]\n[City, State, ZIP...",1,1.132379


In [25]:
score_dataset.to_csv("score_dataset.csv", index=False)